In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 数据读取
df_train = pd.read_csv('训练集.csv')  # 训练集文件名
df_val = pd.read_csv('验证集.csv')   # 验证集文件名

# 数据标准化
scaler_coords = StandardScaler()  # 标准化经度、纬度
scaler_value = StandardScaler()   # 标准化 value
df_train[['longitude', 'latitude']] = scaler_coords.fit_transform(df_train[['longitude', 'latitude']])
df_train['value'] = scaler_value.fit_transform(df_train[['value']])

df_val[['longitude', 'latitude']] = scaler_coords.transform(df_val[['longitude', 'latitude']])  # 用训练集的标准化器
df_val['value'] = scaler_value.transform(df_val[['value']])  # 用训练集的标准化器

# 提取输入和目标值
X_train = df_train[['longitude', 'latitude']].values
y_train = df_train['value'].values

X_val = df_val[['longitude', 'latitude']].values
y_val = df_val['value'].values

# 将数据转换为 PyTorch Tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device)

# 定义 GRU 模型
class GRUPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GRUPredictor, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# 超参数设置
input_dim = 2  # 输入特征维度: longitude 和 latitude
hidden_dim = 64  # GRU 隐层维度
output_dim = 1  # 输出维度: value
num_layers = 3  # GRU 层数
epochs = 2000  # 训练轮次
learning_rate = 0.001  # 学习率

# 初始化 GRU 模型
model = GRUPredictor(input_dim, hidden_dim, output_dim, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 模型训练
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}")

# 模型验证
model.eval()
with torch.no_grad():
    y_pred_val = model(X_val_tensor).cpu().numpy()
    y_val_actual = y_val_tensor.cpu().numpy()

# 反标准化处理
predictions_original = scaler_value.inverse_transform(y_pred_val)
actual_values_original = scaler_value.inverse_transform(y_val_actual)
coords_val_original = scaler_coords.inverse_transform(X_val)

# 保存结果
df_result = pd.DataFrame({
    'longitude': np.round(coords_val_original[:, 0], 6),
    'latitude': np.round(coords_val_original[:, 1], 6),
    'actual_value': np.round(actual_values_original.flatten(), 2),
    'predicted_value': np.round(predictions_original.flatten(), 2)
})

df_result.to_csv('predictions-gru.csv', index=False)
print("训练完成，结果已保存至 'predictions-gru.csv'")


Using device: cpu
Epoch [20/2000], Loss: 0.983982
Epoch [40/2000], Loss: 0.966633
Epoch [60/2000], Loss: 0.952057
Epoch [80/2000], Loss: 0.922590
Epoch [100/2000], Loss: 0.878331
Epoch [120/2000], Loss: 0.845706
Epoch [140/2000], Loss: 0.825468
Epoch [160/2000], Loss: 0.803782
Epoch [180/2000], Loss: 0.750030
Epoch [200/2000], Loss: 0.671762
Epoch [220/2000], Loss: 0.627460
Epoch [240/2000], Loss: 0.557614
Epoch [260/2000], Loss: 0.441442
Epoch [280/2000], Loss: 0.348936
Epoch [300/2000], Loss: 0.301305
Epoch [320/2000], Loss: 0.265435
Epoch [340/2000], Loss: 0.235128
Epoch [360/2000], Loss: 0.209758
Epoch [380/2000], Loss: 0.190649
Epoch [400/2000], Loss: 0.173111
Epoch [420/2000], Loss: 0.154621
Epoch [440/2000], Loss: 0.137867
Epoch [460/2000], Loss: 0.125641
Epoch [480/2000], Loss: 0.117034
Epoch [500/2000], Loss: 0.110900
Epoch [520/2000], Loss: 0.106427
Epoch [540/2000], Loss: 0.101473
Epoch [560/2000], Loss: 0.096935
Epoch [580/2000], Loss: 0.092380
Epoch [600/2000], Loss: 0.087